# Reshape raw weather data
* Weather data is sourced from weather GUI on Orchard House machine

### Load and format raw export

In [1]:
import pandas as pd

In [2]:
full_src = '../src/orchard_house/orchard_house-weather_export-full_history_001-190422.csv'

In [5]:
df = pd.read_csv(full_src, parse_dates=['Date Time '])

In [12]:
# remove columns
df = df.drop(columns=['Day of the Week '])

In [16]:
# remove space from 'Date Time ' column
df = df.rename(index=str, columns={"Date Time ": "datetime"})

### Reshape DataFrame

In [20]:
# Unpivots a DataFrame from wide format to long format
df = pd.melt(df.reset_index()
    , id_vars=['datetime']
    , value_vars=['Outside Temperature TX1 (°F)',
       'High Outside Temperature TX1 (°F)', 'Low Outside Temperature TX1 (°F)',
       'Wind Speed TX1 (mph)', 'High Wind Speed TX1 (mph)',
       'Rain Amount TX1 (in)', 'High Rain Rate TX1 (in/hr)',
       'Outside Temperature TX2 (°F)', 'High Outside Temperature TX2 (°F)',
       'Low Outside Temperature TX2 (°F)', 'Wind Speed TX2 (mph)',
       'High Wind Speed TX2 (mph)', 'Rain Amount TX2 (in)',
       'High Rain Rate TX2 (in/hr)', 'Outside Temperature TX3 (°F)',
       'High Outside Temperature TX3 (°F)', 'Low Outside Temperature TX3 (°F)',
       'Wind Speed TX3 (mph)', 'High Wind Speed TX3 (mph)',
       'Rain Amount TX3 (in)', 'High Rain Rate TX3 (in/hr)',
       'Outside Temperature TX4 (°F)', 'High Outside Temperature TX4 (°F)',
       'Low Outside Temperature TX4 (°F)', 'Wind Speed TX4 (mph)',
       'High Wind Speed TX4 (mph)', 'Rain Amount TX4 (in)',
       'High Rain Rate TX4 (in/hr)', 'Outside Temperature TX5 (°F)',
       'High Outside Temperature TX5 (°F)', 'Low Outside Temperature TX5 (°F)',
       'Wind Speed TX5 (mph)', 'High Wind Speed TX5 (mph)',
       'Rain Amount TX5 (in)', 'High Rain Rate TX5 (in/hr)',
       'Outside Temperature TX6 (°F)', 'High Outside Temperature TX6 (°F)',
       'Low Outside Temperature TX6 (°F)', 'Wind Speed TX6 (mph)',
       'High Wind Speed TX6 (mph)', 'Rain Amount TX6 (in)',
       'High Rain Rate TX6 (in/hr)'])

### Translate row weather station ID to colloquial name

In [25]:
# Add 'location_name' column colloquial convention
df.loc[df['variable'].str.contains('TX1'), 'location_name'] = 'baldy summit'
df.loc[df['variable'].str.contains('TX2'), 'location_name'] = 'baldy draw'
df.loc[df['variable'].str.contains('TX3'), 'location_name'] = 'sanfoin bench'
df.loc[df['variable'].str.contains('TX4'), 'location_name'] = 'south baldy ridge'
df.loc[df['variable'].str.contains('TX5'), 'location_name'] = 'orchard house'
df.loc[df['variable'].str.contains('TX6'), 'location_name'] = 'indian ridge'

### remove TX# values
* this will allow the columns to be combined

In [29]:
val_arr = [' TX1', ' TX2', ' TX3', ' TX4', ' TX5', ' TX6']
for val in val_arr:
    df['variable'] = df.variable.replace({val:''}, regex=True)

### Unstack variable column for legibility

In [40]:
df = df.set_index(['location_name', 'datetime', 'variable'])['value'].unstack()
df = new_df.sort_index()

### Reset index for export

In [57]:
df = df.reset_index().set_index('datetime').sort_index()

In [64]:
df.tail()

variable,location_name,High Outside Temperature (°F),High Rain Rate (in/hr),High Wind Speed (mph),Low Outside Temperature (°F),Outside Temperature (°F),Rain Amount (in),Wind Speed (mph)
datetime,,,,,,,,
2019-04-21 17:00:00,orchard house,49.6,0.0,8.0,48.7,49.6,0.0,2.3
2019-04-21 17:00:00,baldy summit,41.4,0.0,9.0,39.6,41.3,0.0,5.4
2019-04-21 17:00:00,baldy draw,46.5,0.0,12.0,44.9,46.4,0.0,7.1
2019-04-21 17:00:00,sanfoin bench,50.0,0.0,9.0,48.4,49.9,0.0,2.6
2019-04-21 17:00:00,south baldy ridge,45.5,0.0,8.0,43.2,45.5,0.0,3.4


### Export

In [67]:
export_path = '../output/orchard_house/full/reshape/orchard_house-weather_archive-190422.csv'
df.to_csv(export_path)